In [ ]:
import numpy as np
from math import floor

from section4_1_Q import \
    get_Q_T_H_d_t_i, \
    get_Q_T_CS_d_t_i, \
    get_Q_T_CL_d_t_i

from section11_1 import \
    load_outdoor, \
    get_Theta_ex, \
    get_X_ex, \
    get_h_ex

## 第三節　ルームエアコンディショナー

1．適用範囲～4．記号及び単位は省略

### 5. 最大暖房出力

$$\large
Q_{max,H,d,t} = Qr_{max,H,d,t} \times q_{rtd,H} \times C_{af,H} \times C_{df,H,d,t} \times 3600 \times 10^{-6}
$$

$Q_{max,H,d,t}$：日付$d$の時刻$t$における1時間当たりの最大暖房出力(MJ/h)  
$Qr_{max,H,d,t}$：日付$d$の時刻$t$における最大暖房出力比  
$q_{rtd,H}$：定格暖房能力(W)  
$C_{af,H}$：室内機吹き出し風量に関する暖房出力補正係数  
$C_{df,H,d,t}$：日付$d$の時刻$t$におけるデフロストに関する暖房出力補正係数  

In [ ]:
# 最大暖房出力 (1)
def get_Q_max_H_d_t(Q_r_max_H_d_t, q_rtd_H, Theta_ex, h_ex):
    # 室内機吹き出し風量に関する暖房時の能力補正係
    C_af_H = get_C_af_H()

    # デフロストに関する暖房出力補正係数
    C_df_H_d_t = get_C_df_H(Theta_ex, h_ex)

    return Q_r_max_H_d_t * q_rtd_H * C_af_H * C_df_H_d_t * 3600 * 10 ** (-6)

$$\large
Qr_{max,H,d,t} = a_2 \times (\theta_{ex,d,t} - 7)^2 + a_1 \times (\theta_{ex,d,t} - 7) + a_0
$$

$\theta_{ex,d,t}$：日付$d$の時刻$t$における外気温度(℃)  

In [ ]:
# 最大暖房出力比 (2)
def get_Q_r_max_H_d_t(q_rtd_C, q_r_max_H, Theta_ex_d_t):
    # 係数a2及びa2,a0
    a2, a1, a0 = get_a_eq3(q_r_max_H, q_rtd_C)

    return a2 * (Theta_ex_d_t - 7) ** 2 + a1 * (Theta_ex_d_t - 7) + a0

$q_{rtd,C}$が5600(W)を超える場合については、$q_{rtd,C}$は5600(W)に等しいとして計算するものとする。

$$
\Biggl(\begin{matrix}
    a_2\\
    a_1\\
    a_0\\
    \end{matrix}
\Biggr) = 
\Biggl(\begin{matrix}
    b_2\\ 
    b_1\\ 
    b_0\\ 
    \end{matrix}
\Biggr) \times qr_{max,H} + 
\Biggl(\begin{matrix}
    c_2\\ 
    c_1\\ 
    c_0\\ 
    \end{matrix}
\Biggr)
$$

$$
\Biggl(\begin{matrix}
    b_2\\ 
    b_1\\ 
    b_0\\ 
    \end{matrix}
\Biggr) = 
\Biggl(\begin{matrix}
    0.000181\\ 
    0.002322\\ 
    0.003556\\ 
    \end{matrix}
\Biggr) \times q_{rtd,C} \times 10^{-3}  + 
\Biggl(\begin{matrix}
    -0.000184\\ 
    0.013904\\ 
    0.993431\\ 
    \end{matrix}
\Biggr)
$$

$$
\Biggl(\begin{matrix}
    c_2\\ 
    c_1\\ 
    c_0\\ 
    \end{matrix}
\Biggr) = 
\Biggl(\begin{matrix}
    -0.000173\\ 
    -0.003980\\ 
    -0.002870\\ 
    \end{matrix}
\Biggr) \times q_{rtd,C} \times 10^{-3}  + 
\Biggl(\begin{matrix}
    0.000367\\ 
    0.003983\\ 
    0.006376\\ 
    \end{matrix}
\Biggr)
$$

$qr_{max,H}$：最大暖房能力比  
$q_{rtd,C}$：定格冷房能力(W)  

In [ ]:
# 係数a2及びa1,a0 (3a)
def get_a_eq3(q_r_max_H, q_rtd_C):
    b2, b1, b0 = get_b_eq3(q_rtd_C)
    c2, c1, c0 = get_c_eq3(q_rtd_C)
    a2 = b2 * q_r_max_H + c2
    a1 = b1 * q_r_max_H + c1
    a0 = b0 * q_r_max_H + c0
    return a2, a1, a0

# 係数b2及びb1,b0 (3b)
def get_b_eq3(q_rtd_C):
    q_rtd_C = min(5600, q_rtd_C)
    b2 = 0.000181 * q_rtd_C * 10 ** (-3) - 0.000184
    b1 = 0.002322 * q_rtd_C * 10 ** (-3) + 0.013904
    b0 = 0.003556 * q_rtd_C * 10 ** (-3) + 0.993431
    return b2, b1, b0


# 係数c2及びc1,c0 (3b)
def get_c_eq3(q_rtd_C):
    q_rtd_C = min(5600, q_rtd_C)
    c2 = -0.000173 * q_rtd_C * 10 ** (-3) + 0.000367
    c1 = -0.003980 * q_rtd_C * 10 ** (-3) + 0.003983
    c0 = -0.002870 * q_rtd_C * 10 ** (-3) + 0.006376
    return c2, c1, c0

$$\large
qr_{max,H} = \frac{q_{max,H}}{q_{rtd,H}}
$$

$q_{max,H}$：最大暖房能力(W)  
$q_{rtd,H}$：定格暖房能力(W)  

In [ ]:
# 最大暖房能力比 (4)
def get_q_r_max_H(q_max_H, q_rtd_H):
    q_r_max_H = q_max_H / q_rtd_H
    return q_r_max_H

室内機吹き出し風量に関する暖房時の能力補正係数$C_{af,H}$は0.8とする。  
デフロストに関する暖房出力補正係数$C_{df,H}$は、外気温度が5℃未満かつ相対湿度が80%以上の場合にデフロストが入ると仮定し、その場合の値を0.77とし、 それ以外の条件においては1.0とする。

In [ ]:
# 室内機吹き出し風量に関する暖房時の能力補正係
def get_C_af_H():
    return 0.8


# デフロストに関する暖房出力補正係数
def get_C_df_H(Theta_ex, h_ex):
    C_df_H = np.ones(24 * 365)
    C_df_H[(Theta_ex < 5.0) * (h_ex >= 80.0)] = 0.77
    return C_df_H

### 6. 暖房エネルギー消費量

#### 6.1 消費電力量

日付$d$の時刻$t$における補正処理暖房負荷$Q'_{T,H,d,t}$が0の場合、$E_{E,H,d,t}$ = 0とする。

$$\large
E_{E,H,d,t} = f_{H,\theta} \Biggl( \frac{Q'_{T,H,d,t}}{q_{max,H} \times 3600 \times 10^{-6}} \Biggr) \div 
f_{H,7} \Biggl( \frac{1}{qr_{max,H}} \Biggr) \times \frac{q_{rtd,H}}{e_{rtd,H}} \times 10^{-3}
$$

$E_{E,H,d,t}$：日付$d$の時刻$t$における1時間当たりの消費電力量(kWh/h)  
$f_{H,\theta}$：基準入出力関数  
$Q'_{T,H,d,t}$：日付$d$の時刻$t$における補正処理暖房負荷(MJ/h)    
$q_{max,H}$：最大暖房能力(W)  
$qr_{max,H}$：最大暖房能力比  
$q_{rtd,H}$：定格暖房能力(W)  
$e_{rtd,H}$：定格暖房エネルギー消費効率  

In [ ]:
# 消費電力量 (5)
# dualcompressor: 容量可変型コンプレッサー搭載
def get_E_E_H_d_t(region, q_rtd_C, q_rtd_H, e_rtd_H, dualcompressor, L_H_d_t):
    # 外気条件
    outdoor = load_outdoor()
    Theta_ex = get_Theta_ex(region, outdoor)
    X_ex = get_X_ex(region, outdoor)
    h_ex = get_h_ex(X_ex, Theta_ex)

    # 最大暖房能力
    q_max_C = get_q_max_C(q_rtd_C)
    q_max_H = get_q_max_H(q_rtd_H, q_max_C)

    # 最大暖房能力比
    q_r_max_H = get_q_r_max_H(q_max_H, q_rtd_H)

    # 最大暖房出力比
    Q_r_max_H_d_t = get_Q_r_max_H_d_t(q_rtd_C, q_r_max_H, Theta_ex)

    # 最大暖房出力
    Q_max_H_d_t = get_Q_max_H_d_t(Q_r_max_H_d_t, q_rtd_H, Theta_ex, h_ex)

    # 処理暖房負荷
    Q_T_H_d_t = get_Q_T_H_d_t_i(Q_max_H_d_t, L_H_d_t, None, '居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合に該当しない場合')

    # 補正処理暖房負荷
    Q_dash_T_H_d_t = get_Q_dash_T_H_d_t(Q_T_H_d_t, Theta_ex, h_ex)

    # 消費電力量
    E_E_H_d_t = f_H_Theta(Q_dash_T_H_d_t / (q_max_H * 3600 * 10 ** (-6)), q_rtd_C, dualcompressor, Theta_ex) \
                / f_H_Theta(1.0 / q_r_max_H, q_rtd_C, dualcompressor, np.ones(24 * 365) * 7.0) \
                * (q_rtd_H / e_rtd_H) * 10 ** (-3)
    E_E_H_d_t[Q_dash_T_H_d_t == 0.0] = 0.0  # 補正処理暖房負荷が0の場合は0

    return E_E_H_d_t


def get_Q_UT_H_d_t(region, q_rtd_C, q_rtd_H, e_rtd_H, L_H_d_t):
    # 外気条件
    outdoor = load_outdoor()
    Theta_ex = get_Theta_ex(region, outdoor)
    X_ex = get_X_ex(region, outdoor)
    h_ex = get_h_ex(X_ex, Theta_ex)

    # 最大暖房能力
    q_max_C = get_q_max_C(q_rtd_C)
    q_max_H = get_q_max_H(q_rtd_H, q_max_C)

    # 最大暖房能力比
    q_r_max_H = get_q_r_max_H(q_max_H, q_rtd_H)

    # 最大暖房出力比
    Q_r_max_H_d_t = get_Q_r_max_H_d_t(q_rtd_C, q_r_max_H, Theta_ex)

    # 最大暖房出力
    Q_max_H_d_t = get_Q_max_H_d_t(Q_r_max_H_d_t, q_rtd_H, Theta_ex, h_ex)

    # 処理暖房負荷
    Q_T_H_d_t = get_Q_T_H_d_t_i(Q_max_H_d_t, L_H_d_t, None, '居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合に該当しない場合')

    return L_H_d_t - Q_T_H_d_t

$$\large
f_{H,\theta}(x) = a_{4}x^{4} + a_{3}x^{3} + a_{2}x^{2} + a_{1}x + a_{0}
$$

In [ ]:
 基準入出力関数 (6)
# dualcompressor: 容量可変型コンプレッサー搭載
def f_H_Theta(x, q_rtd_C, dualcompressor, Theta_ex):
    # 係数a0-a4
    a0, a1, a2, a3, a4 = get_a_eq7(q_rtd_C, dualcompressor, Theta_ex)
    return a4 * x ** 4 + a3 * x ** 3 + a2 * x ** 2 + a1 * x + a0

\begin{equation*}
\left(\begin{matrix}
    a_4\\
    a_3\\
    a_2\\
    a_1\\
    a_0\\
    \end{matrix}
\right) = 
\left(\begin{matrix}
    p_{42} & p_{41} & p_{40}\\
    p_{32} & p_{31} & p_{30}\\
    p_{22} & p_{21} & p_{20}\\
    p_{12} & p_{11} & p_{10}\\
    p_{02} & p_{01} & p_{00}\\
    \end{matrix}
\right)
\left(\begin{matrix}
    \theta_{ex}^2\\ 
    \theta_{ex}\\ 
    1\\ 
    \end{matrix}
\right)
\end{equation*}

In [ ]:
# 係数a0～a4 (7)
# dualcompressor: 容量可変型コンプレッサー搭載
def get_a_eq7(q_rtd_C, dualcompressor, Theta_ex):
    if dualcompressor == False:
        # 容量可変型コンプレッサー搭載ルームエアコンディショナーでないルームエアコンディショナー
        get_p_i = get_p_i_eq8
    else:
        # 容量可変型コンプレッサー搭載ルームエアコンディショナー
        get_p_i = get_p_i_eq9

    # 係数p_i
    p_42 = get_p_i(42, q_rtd_C)
    p_41 = get_p_i(41, q_rtd_C)
    p_40 = get_p_i(40, q_rtd_C)
    p_32 = get_p_i(32, q_rtd_C)
    p_31 = get_p_i(31, q_rtd_C)
    p_30 = get_p_i(30, q_rtd_C)
    p_22 = get_p_i(22, q_rtd_C)
    p_21 = get_p_i(21, q_rtd_C)
    p_20 = get_p_i(20, q_rtd_C)
    p_12 = get_p_i(12, q_rtd_C)
    p_11 = get_p_i(11, q_rtd_C)
    p_10 = get_p_i(10, q_rtd_C)
    p_02 = get_p_i(2, q_rtd_C)
    p_01 = get_p_i(1, q_rtd_C)
    p_00 = get_p_i(0, q_rtd_C)

    a4 = p_42 * Theta_ex ** 2 + p_41 * Theta_ex + p_40 * 1
    a3 = p_32 * Theta_ex ** 2 + p_31 * Theta_ex + p_30 * 1
    a2 = p_22 * Theta_ex ** 2 + p_21 * Theta_ex + p_20 * 1
    a1 = p_12 * Theta_ex ** 2 + p_11 * Theta_ex + p_10 * 1
    a0 = p_02 * Theta_ex ** 2 + p_01 * Theta_ex + p_00 * 1

    return a0, a1, a2, a3, a4

$q_{rtd,C}$が5600(W)を超える場合については、$q_{rtd,C}$は5600(W)に等しいとして計算するものとする。

\begin{equation*}
\large
p_{i} = s_{i} \times q_{rtd,C} \times 10^{-3} + t_{i}
\end{equation*}

$q_{rtd,C}$：定格冷房能力(W)  

In [ ]:
# 係数p_i (8) (i=0,1,2,10..42)
# (容量可変型コンプレッサー搭載ルームエアコンディショナーでないルームエアコンディショナー)
def get_p_i_eq8(i, q_rtd_C):
    q_rtd_C = min(5600, q_rtd_C)
    # 係数 s_i, t_i
    s_i = get_s_i_eq8(i)
    t_i = get_t_i_eq8(i)
    return s_i * q_rtd_C * 10 ** (-3) + t_i


# 係数s_i (i=0,1,2,10..42)
def get_s_i_eq8(i):
    return table_3[4 - floor(i / 10)][(2 - (i % 10)) * 2]


# 係数t_i (i=0,1,2,10..42)
def get_t_i_eq8(i):
    return table_3[4 - floor(i / 10)][(2 - (i % 10)) * 2 + 1]

<div style="text-align: center;font-weight: bold;">表　係数 $s_{i}$ 及び $t_{i}$</div>

|係数|値|係数|値|係数|値|係数|値|係数|値|係数|値|
|-|-|-|-|-|-|-|-|-|-|-|-|
|$s_{42}$|-0.00236|$t_{42}$|0.01324|$s_{41}$|0.08418|$t_{41}$|-0.47143|$s_{40}$|-1.16944|$t_{40}$|6.54886|
|$s_{32}$|0.00427|$t_{32}$|-0.02392|$s_{31}$|-0.19226|$t_{31}$|0.94213|$s_{30}$|2.58632|$t_{30}$|-12.85618|
|$s_{22}$|-0.00275|$t_{22}$|0.01542|$s_{21}$|0.14947|$t_{21}$|-0.68303|$s_{20}$|-2.03594|$t_{20}$|10.60561|
|$s_{12}$|0.00063|$t_{12}$|-0.00351|$s_{11}$|-0.02865|$t_{11}$|0.10522|$s_{10}$|0.37336|$t_{10}$|-1.09499|
|$s_{02}$|-0.00005|$t_{02}$|0.00028|$s_{01}$|0.00184|$t_{01}$|-0.01090|$s_{00}$|-0.09609|$t_{00}$|0.59229|


In [ ]:
# 表3 係数s_i及びt_i
table_3 = [
    (-0.00236, 0.01324, 0.08418, -0.47143, -1.16944, 6.54886),
    (0.00427, -0.02392, -0.19226, 0.94213, 2.58632, -12.85618),
    (-0.00275, 0.01542, 0.14947, -0.68303, -2.03594, 10.60561),
    (0.00063, -0.00351, -0.02865, 0.10522, 0.37336, -1.09499),
    (-0.00005, 0.00028, 0.00184, -0.01090, -0.09609, 0.59229)
]

$q_{rtd,C} \leq 2200$ の場合

$$\large
p_{i} = p_{i,A}
$$

$2200 < q_{rtd,C} \leq 4000$ の場合

$$\large
p_{i} = p_{i,A} \times \frac{4000-q_{rtd,C}}{4000-2200} + p_{i,B} \times \frac{q_{rtd,C}-2200}{4000-2200}
$$

$4000 < q_{rtd,C} < 7100$ の場合

$$\large
p_{i} = p_{i,B} \times \frac{7100-q_{rtd,C}}{7100-4000} + p_{i,C} \times \frac{q_{rtd,C}-4000}{7100-4000}
$$

$7100 \leq q_{rtd,C}$ の場合

$$\large
p_{i} = p_{i,C}
$$

$p_{i,A}$：表により表される係数  
$p_{i,B}$：表により表される係数  
$p_{i,C}$：表により表される係数  

In [ ]:
# 係数p_i (9)
# (容量可変型コンプレッサー搭載ルームエアコンディショナー)
def get_p_i_eq9(i, q_rtd_C):
    if q_rtd_C <= 2200:
        # (9a)
        return get_p_i_A(i)
    elif 2200 < q_rtd_C and q_rtd_C <= 4000:
        # (9b)
        p_i_A = get_p_i_A(i)
        p_i_B = get_p_i_B(i)
        return p_i_A * ((4000 - q_rtd_C) / (4000 - 2200)) \
               + p_i_B * ((q_rtd_C - 2200) / (4000 - 2200))
    elif 4000 < q_rtd_C and q_rtd_C < 7100:
        # (9c)
        p_i_B = get_p_i_B(i)
        p_i_C = get_p_i_C(i)
        return p_i_B * ((7100 - q_rtd_C) / (7100 - 4000)) \
               + p_i_C * ((q_rtd_C - 4000) / (7100 - 4000))
    elif 7100 <= q_rtd_C:
        return get_p_i_C(i)
    else:
        raise ValueError(q_rtd_C)


# 係数P_i_A (i=0,1,2,10..42)
def get_p_i_A(i):
    return table_4_A[4 - floor(i / 10)][(2 - (i % 10))]


def get_p_i_B(i):
    return table_4_B[4 - floor(i / 10)][(2 - (i % 10))]


def get_p_i_C(i):
    return table_4_C[4 - floor(i / 10)][(2 - (i % 10))]

<div style="text-align: center;font-weight: bold;">表　係数$p_{i,A}$</div>

|係数|値|係数|値|係数|値|
|-|-|-|-|-|-|
|$p_{42}$|-0.000056|$p_{41}$|0.000786|$p_{40}$|0.071625|
|$p_{32}$|-0.000145|$p_{31}$|0.003337|$p_{30}$|-0.143643|
|$p_{22}$|-0.000240|$p_{21}$|-0.029471|$p_{20}$|1.954343|
|$p_{12}$|-0.000035|$p_{11}$|-0.050909|$p_{10}$|1.389751|
|$p_{02}$|0.0|$p_{01}$|0.0|$p_{00}$|0.076800|  


In [ ]:
# 表4(A) 係数 p_i_A
table_4_A = [
    (-0.000056, 0.000786, 0.071625),
    (-0.000145, 0.003337, -0.143643),
    (-0.000240, -0.029471, 1.954343),
    (-0.000035, -0.050909, 1.389751),
    (0.0, 0.0, 0.076800)
]

<div style="text-align: center;font-weight: bold;">表　係数$p_{i,B}$</div>

|係数|値|係数|値|係数|値|
|-|-|-|-|-|-|
|$p_{42}$|0.000108|$p_{41}$|-0.035658|$p_{40}$|3.063873|
|$p_{32}$|-0.000017|$p_{31}$|0.062546|$p_{30}$|-5.471556|
|$p_{22}$|-0.000245|$p_{21}$|-0.025126|$p_{20}$|4.057590|
|$p_{12}$|0.000323|$p_{11}$|-0.021166|$p_{10}$|0.575459|
|$p_{02}$|0.0|$p_{01}$|0.000330|$p_{00}$|0.047500| 


In [ ]:
# 表4(B) 係数 p_i_B
table_4_B = [
    (0.000108, -0.035658, 3.063873),
    (-0.000017, 0.062546, -5.471556),
    (-0.000245, -0.025126, 4.057590),
    (0.000323, -0.021166, 0.575459),
    (0.0, 0.000330, 0.047500)
]

<div style="text-align: center;font-weight: bold;">表　係数$p_{i,C}$</div>

|係数|値|係数|値|係数|値|
|-|-|-|-|-|-|
|$p_{42}$|-0.001465|$p_{41}$|-0.030500|$p_{40}$|1.920431|
|$p_{32}$|0.002824|$p_{31}$|0.041081|$p_{30}$|-1.835302|
|$p_{22}$|-0.001929|$p_{21}$|-0.009738|$p_{20}$|1.582898|
|$p_{12}$|0.000616|$p_{11}$|-0.014239|$p_{10}$|0.546204|
|$p_{02}$|0.0|$p_{01}$|-0.000110|$p_{00}$|0.023100|


In [ ]:
# 表4(C) 係数 p_i_C
table_4_C = [
    (-0.001465, -0.030500, 1.920431),
    (0.002824, 0.041081, -1.835302),
    (-0.001929, -0.009738, 1.582898),
    (0.000616, -0.014239, 0.546204),
    (0.0, -0.000110, 0.023100)
]

$$\large
Q'_{T,H,d,t} = Q_{T,H,d,t} \times \frac{1}{C_{af,H} \times C_{df}}
$$

$Q_{T,H,d,t}$：日付$d$の時刻$t$における1時間当たりの暖房設備機器の処理暖房負荷(MJ/h)

In [ ]:
# 補正処理暖房負荷 (10)
def get_Q_dash_T_H_d_t(Q_T_H_d_t, Theta_ex, h_ex):
    ##室内機吹き出し風量に関する暖房時の能力補正係
    C_af_H = get_C_af_H()

    ##デフロストに関する暖房出力補正係数
    C_d_f = get_C_df_H(Theta_ex, h_ex)

    Q_dash_T_H_d_t = Q_T_H_d_t * (1.0 / (C_af_H * C_d_f))

    return Q_dash_T_H_d_t

#### 6.2 ガス消費量

日付$d$の時刻$t$における１時間当たりのガス消費量$E_{G,H,d,t}$は0とする。

In [ ]:
# ガス消費量
def get_E_G_H_d_t():
    return np.zeros(24 * 365)

#### 6.3 灯油消費量

日付$d$の時刻$t$における１時間当たりの灯油消費量$E_{K,H,d,t}$は0とする。

In [ ]:
# 灯油消費量
def get_E_K_H_d_t():
    return np.zeros(24 * 365)

#### 6.4 その他の燃料による一次エネルギー消費量

日付$d$の時刻$t$における１時間当たりのその他の燃料による一次エネルギー消費量$E_{M,H,d,t}$は0とする。

In [ ]:
# その他の燃料による一次エネルギー消費量
def get_E_M_H_d_t():
    return np.zeros(24 * 365)

### 7. 最大冷房出力

#### 7.1 最大冷房出力の計算

$$\large
Q_{max,C,d,t} = Qr_{max,C,d,t} \times q_{rtd,C} \times C_{af,C} \times C_{hm,C} \times 3600 \times 10^{-6}
$$

$Q_{max,C,d,t}$：日付$d$の時刻$t$における1時間当たりの最大冷房出力(MJ/h)  
$Qr_{max,C,d,t}$：日付$d$の時刻$t$における最大冷房出力比  
$q_{rtd,C}$：定格冷房能力(W)  
$C_{af,C}$：室内機吹き出し風量に関する冷房出力補正係数  
$C_{hm,C}$：室内機吸い込み湿度に関する冷房出力補正係数  

In [ ]:
# 最大冷房出力 (11)
def get_Q_max_C_d_t(Q_r_max_C_d_t, q_rtd_C):
    # 室内機吸い込み湿度に関する冷房出力補正係数
    C_hm_C = get_C_hm_C()

    # 室内機吹き出し風量に関する冷房出力係数
    C_af_C = get_C_af_C()

    return Q_r_max_C_d_t * q_rtd_C * C_af_C * C_hm_C * 3600 * 10 ** (-6)

$$\large
Qr_{max,C,d,t} = a_2 \times (\theta_{ex,d,t} - 35)^2 + a_1 \times (\theta_{ex,d,t} - 35) + a_0
$$

In [ ]:
# 最大冷房出力比 (12)
def get_Q_r_max_C_d_t(q_r_max_C, q_rtd_C, Theta_ex_d_t):
    a2, a1, a0 = get_a_eq13(q_r_max_C, q_rtd_C)
    return a2 * (Theta_ex_d_t - 35) ** 2 + a1 * (Theta_ex_d_t - 35) + a0

$q_{rtd,C}$が5600(W)を超える場合については、$q_{rtd,C}$は5600(W)に等しいとして計算するものとする。

$$
\Biggl(\begin{matrix}
    a_2\\
    a_1\\
    a_0\\
    \end{matrix}
\Biggr) = 
\Biggl(\begin{matrix}
    b_2\\ 
    b_1\\ 
    b_0\\ 
    \end{matrix}
\Biggr) \times qr_{max,C} + 
\Biggl(\begin{matrix}
    c_2\\ 
    c_1\\ 
    c_0\\ 
    \end{matrix}
\Biggr)
$$

$$
\Biggl(\begin{matrix}
    b_2\\ 
    b_1\\ 
    b_0\\ 
    \end{matrix}
\Biggr) = 
\Biggl(\begin{matrix}
    0.000182\\ 
    0.003527\\ 
    -0.011490\\ 
    \end{matrix}
\Biggr) \times q_{rtd,C} \times 10^{-3}  + 
\Biggl(\begin{matrix}
    -0.001480\\ 
    -0.023000\\ 
    1.024328\\ 
    \end{matrix}
\Biggr)
$$

$$
\Biggl(\begin{matrix}
    c_2\\ 
    c_1\\ 
    c_0\\ 
    \end{matrix}
\Biggr) = 
\Biggl(\begin{matrix}
    -0.000350\\ 
    -0.001280\\ 
    0.004772\\ 
    \end{matrix}
\Biggr) \times q_{rtd,C} \times 10^{-3}  + 
\Biggl(\begin{matrix}
    0.000800\\ 
    0.003621\\ 
    -0.011170\\ 
    \end{matrix}
\Biggr)
$$

$qr_{max,C}$：最大冷房能力比  
$q_{rtd,C}$：定格冷房能力(W)  

In [ ]:
# 係数a2及びa1,a0 (13a)
def get_a_eq13(q_r_max_C, q_rtd_C):
    b2, b1, b0 = get_b_eq13(q_rtd_C)
    c2, c1, c0 = get_c_eq13(q_rtd_C)
    a2 = b2 * q_r_max_C + c2
    a1 = b1 * q_r_max_C + c1
    a0 = b0 * q_r_max_C + c0
    return a2, a1, a0


# 係数b2,b1,b0 (13b)
def get_b_eq13(q_rtd_C):
    q_rtd_C = min(5600, q_rtd_C)
    b2 = 0.000812 * q_rtd_C * 10 ** (-3) - 0.001480
    b1 = 0.003527 * q_rtd_C * 10 ** (-3) - 0.023000
    b0 = -0.011490 * q_rtd_C * 10 ** (-3) + 1.024328
    return b2, b1, b0


# 係数c2,c1,c0 (13c)
def get_c_eq13(q_rtd_C):
    q_rtd_C = min(5600, q_rtd_C)
    c2 = -0.000350 * q_rtd_C * 10 ** (-3) + 0.000800
    c1 = -0.001280 * q_rtd_C * 10 ** (-3) + 0.003621
    c0 = 0.004772 * q_rtd_C * 10 ** (-3) - 0.011170
    return c2, c1, c0

$$\large
qr_{max,C} = \frac{q_{max,C}}{q_{rtd,C}}
$$

$q_{max,C}$：最大冷房能力(W)  
$q_{rtd,C}$：定格冷房能力(W)  

In [ ]:
# 最大冷房能力比 (14)
def get_q_r_max_C(q_max_C, q_rtd_C):
    return q_max_C / q_rtd_C

室内機吹き出し風量に関する冷房時の能力補正係数$C_{af,C}$は0.85とする。  
室内機吸い込み湿度に関する冷房能力補正係数$C_{hm,C}$は1.15とする。

In [ ]:
# 室内機吹き出し風量に関する冷房時の能力補正係数 C_af_C
def get_C_af_C():
    return 0.85


# 室内機吸い込み湿度に関する冷房能力補正係 C_hm_C
def get_C_hm_C():
    return 1.15

#### 7.2 最大冷房顕熱出力及び最大冷房潜熱出力の計算

$$\large
Q_{max,CS,d,t} = Q_{max,C,d,t} \times SHF'_{d,t}
$$

$$\large
Q_{max,CL,d,t} = min(Q_{max,C,d,t} \times (1 - SHF'_{d,t}),  L'_{CL,d,t})
$$

$Q_{max,CS,d,t}$：日付$d$の時刻$t$における1時間当たりの最大冷房顕熱出力(MJ/h)  
$Q_{max,CL,d,t}$：日付$d$の時刻$t$における1時間当たりの最大冷房潜熱出力(MJ/h)  
$Q_{max,C,d,t}$：日付$d$の時刻$t$における1時間当たりの最大冷房出力(MJ/h)  
$SHF'_{d,t}$：日付$d$の時刻$t$における冷房負荷補正顕熱比  

In [ ]:
# 最大冷房顕熱出力 (15a)
def get_Q_max_CS_d_t(Q_max_C_d_t, SHF_dash_d_t):
    return Q_max_C_d_t * SHF_dash_d_t


# 最大冷房潜熱出力 (15b)
def get_Q_max_CL_d_t(Q_max_C_d_t, SHF_dash_d_t, L_dash_CL_d_t):
    return np.clip(Q_max_C_d_t * (1.0 - SHF_dash_d_t), 0, L_dash_CL_d_t)

$$\large
SHF'_{d,t} = \left\{ \begin{array}{ll}
    \frac{L_{CS,d,t}}{L'_{C,d,t}} & (L'_{C,d,t} \neq 0)\\ 
    0 & (L'_{C,d,t} = 0)\\ 
    \end{array}
\right.
$$

$L_{CS,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房顕熱負荷(MJ/h)  
$L'_{C,d,t}$：日付$d$の時刻$t$における1時間当たりの補正冷房負荷(MJ/h)  

In [ ]:
# 冷房負荷補正顕熱比 (16)
def get_SHF_dash_d_t(L_CS_d_t, L_dash_C_d_t):
    SHF_dash_d_t = np.zeros(24*365)
    f1 = L_dash_C_d_t > 0
    SHF_dash_d_t[f1] = L_CS_d_t[f1] / L_dash_C_d_t[f1]
    return SHF_dash_d_t

$$\large
L'_{C,d,t} = L_{CS,d,t} + L'_{CL,d,t}
$$

$L_{CS,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房顕熱負荷(MJ/h)  
$L'_{CL,d,t}$：日付$d$の時刻$t$における1時間当たりの補正冷房潜熱負荷(MJ/h)  

In [ ]:
# 補正冷房負荷 (17)
def get_L_dash_C_d_t(L_CS_d_t, L_dash_CL_d_t):
    return L_CS_d_t + L_dash_CL_d_t

$$\large
L'_{CL,d,t} = min(L_{max,CL,d,t},  L_{CL,d,t})
$$

$L_{max,CL,d,t}$：日付$d$の時刻$t$における1時間当たりの最大冷房潜熱負荷(MJ/h)  
$L_{CL,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房潜熱負荷(MJ/h)  

In [ ]:
# 補正冷房潜熱負荷 (18)
def get_L_dash_CL_d_t(L_max_CL_d_t, L_CL_d_t):
    return np.clip(L_CL_d_t, 0, L_max_CL_d_t)

$$\large
L_{max,CL,d,t} = L_{CS,d,t} \times \frac{1 - SHF_{L,min,c}}{SHF_{L,min,c}}
$$

$L_{CS,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房顕熱負荷(MJ/h)  
$SHF_{L,min,c}$：冷房負荷最小顕熱比  

In [ ]:
# 最大冷房潜熱負荷 (19)
def get_L_max_CL_d_t(L_CS_d_t, SHF_L_min_c):
    return L_CS_d_t * ((1.0 - SHF_L_min_c) / SHF_L_min_c)

冷房負荷最小顕熱比$SHF_{L,min,c}$は0.4とする。

In [ ]:
# 冷房負荷最小顕熱比
def get_SHF_L_min_c():
    return 0.4

### 8. 冷房エネルギー消費量

#### 8.1 消費電力量

日付$d$の時刻$t$における補正処理冷房負荷 $Q'_{T,c,d,t}$が0の場合、$E_{E,C,d,t}$ = 0とする。

$$\large
E_{E,C,d,t} = f_{C,\theta} \Biggl( \frac{Q'_{T,c,d,t}}{q_{max,C} \times 3600 \times 10^{-6}} \Biggr) \div 
f_{C,35} \Biggl( \frac{1}{qr_{max,C}} \Biggr) \times \frac{q_{rtd,C}}{e_{rtd,C}} \times 10^{-3}
$$

$E_{E,C,d,t}$：日付$d$の時刻$t$における1時間当たりの消費電力量(kWh/h)  
$f_{C,\theta}$：基準入出力関数    
$Q'_{T,c,d,t}$：日付$d$の時刻$t$における補正処理冷房負荷(MJ/h)    
$q_{max,C}$：最大冷房能力(W)    
$q_{rtd,C}$：定格冷房能力(W)    
$e_{rtd,C}$：定格冷房エネルギー消費効率  

In [ ]:
# 消費電力量 (20)
def get_E_E_C_d_t(region, q_rtd_C, e_rtd_C, dualcompressor, L_CS_d_t, L_CL_d_t):
    # 外気条件
    outdoor = load_outdoor()
    Theta_ex = get_Theta_ex(region, outdoor)
    X_ex = get_X_ex(region, outdoor)
    h_ex = get_h_ex(X_ex, Theta_ex)

    # 最大冷房能力
    q_max_C = get_q_max_C(q_rtd_C)

    # 最大冷房能力比
    q_r_max_C = get_q_r_max_C(q_max_C, q_rtd_C)

    # 最大冷房出力比
    Q_r_max_C_d_t = get_Q_r_max_C_d_t(q_r_max_C, q_rtd_C, Theta_ex)

    # 最大冷房出力
    Q_max_C_d_t = get_Q_max_C_d_t(Q_r_max_C_d_t, q_rtd_C)

    # 冷房負荷最小顕熱比
    SHF_L_min_c = get_SHF_L_min_c()

    # 最大冷房潜熱負荷
    L_max_CL_d_t = get_L_max_CL_d_t(L_CS_d_t, SHF_L_min_c)

    # 補正冷房潜熱負荷
    L_dash_CL_d_t = get_L_dash_CL_d_t(L_max_CL_d_t, L_CL_d_t)
    L_dash_C_d_t = get_L_dash_C_d_t(L_CS_d_t, L_dash_CL_d_t)

    # 冷房負荷補正顕熱比
    SHF_dash_d_t = get_SHF_dash_d_t(L_CS_d_t, L_dash_C_d_t)

    # 最大冷房顕熱出力, 最大冷房潜熱出力
    Q_max_CS_d_t = get_Q_max_CS_d_t(Q_max_C_d_t, SHF_dash_d_t)
    Q_max_CL_d_t = get_Q_max_CL_d_t(Q_max_C_d_t, SHF_dash_d_t, L_dash_CL_d_t)

    # 処理冷房負荷
    Q_T_CL_d_t = get_Q_T_CL_d_t_i(Q_max_CL_d_t, L_CL_d_t)
    Q_T_CS_d_t = get_Q_T_CS_d_t_i(Q_max_CS_d_t, L_CS_d_t, None, '居室のみを冷房する方式')

    # 補正処理冷房負荷
    Q_dash_T_C_d_t = get_Q_dash_T_C_d_t(Q_T_CS_d_t, Q_T_CL_d_t)

    # 消費電力量
    E_E_C_d_t = f_C_Theta(Q_dash_T_C_d_t / (q_max_C * 3600 * 10 ** (-6)), Theta_ex, q_rtd_C, dualcompressor) \
                / f_C_Theta(1.0 / q_r_max_C, np.ones(24 * 365) * 35.0, q_rtd_C, dualcompressor) \
                * (q_rtd_C / e_rtd_C) * 10 ** (-3)
    E_E_C_d_t[Q_dash_T_C_d_t == 0.0] = 0.0

    return E_E_C_d_t

$$\large
f_{C,\theta}(x) = a_{4}x^{4} + a_{3}x^{3} + a_{2}x^{2} + a_{1}x + a_{0}
$$

\begin{equation*}
\left(\begin{matrix}
    a_4\\
    a_3\\
    a_2\\
    a_1\\
    a_0\\
    \end{matrix}
\right) = 
\left(\begin{matrix}
    p_{42} & p_{41} & p_{40}\\
    p_{32} & p_{31} & p_{30}\\
    p_{22} & p_{21} & p_{20}\\
    p_{12} & p_{11} & p_{10}\\
    p_{02} & p_{01} & p_{00}\\
    \end{matrix}
\right)
\left(\begin{matrix}
    \theta_{ex}^2\\ 
    \theta_{ex}\\ 
    1\\ 
    \end{matrix}
\right)
\end{equation*}

In [ ]:
# 基準入出力関数 (21)
def f_C_Theta(x, Theta_ex, q_rtd_C, dualcompressor=False):
    # 係数a0-a4
    a0, a1, a2, a3, a4 = get_a_eq22(Theta_ex, q_rtd_C, dualcompressor=dualcompressor)

    return a4 * x ** 4 + a3 * x ** 3 + a2 * x ** 2 + a1 * x + a0


# 係数a0-a4
def get_a_eq22(Theta_ex, q_rtd_C, dualcompressor=False):
    if dualcompressor == False:
        # 容量可変型コンプレッサー搭載ルームエアコンディショナーでないルームエアコンディショナー
        get_p_i = get_p_i_eq23
    else:
        # 容量可変型コンプレッサー搭載ルームエアコンディショナー
        get_p_i = get_p_i_eq24

    # 係数p_i
    p42 = get_p_i(42, q_rtd_C)
    p41 = get_p_i(41, q_rtd_C)
    p40 = get_p_i(40, q_rtd_C)
    p32 = get_p_i(32, q_rtd_C)
    p31 = get_p_i(31, q_rtd_C)
    p30 = get_p_i(30, q_rtd_C)
    p22 = get_p_i(22, q_rtd_C)
    p21 = get_p_i(21, q_rtd_C)
    p20 = get_p_i(20, q_rtd_C)
    p12 = get_p_i(12, q_rtd_C)
    p11 = get_p_i(11, q_rtd_C)
    p10 = get_p_i(10, q_rtd_C)
    p02 = get_p_i(2, q_rtd_C)
    p01 = get_p_i(1, q_rtd_C)
    p00 = get_p_i(0, q_rtd_C)

    a4 = p42 * Theta_ex ** 2 + p41 * Theta_ex + p40 * 1
    a3 = p32 * Theta_ex ** 2 + p31 * Theta_ex + p30 * 1
    a2 = p22 * Theta_ex ** 2 + p21 * Theta_ex + p20 * 1
    a1 = p12 * Theta_ex ** 2 + p11 * Theta_ex + p10 * 1
    a0 = p02 * Theta_ex ** 2 + p01 * Theta_ex + p00 * 1

    return a0, a1, a2, a3, a4

容量可変型コンプレッサー搭載ルームエアコンディショナーでないルームエアコンディショナーについては、係数$p_{i}$は式により与えられ、係数$s_{i}$及び$t_{i}$は表より表される。
$q_{rtd,C}$が5600(W)を超える場合については、$q_{rtd,C}$は5600(W)に等しいとして計算するものとする。

$$
\begin{equation*}
\large
p_{i} = s_{i} \times q_{rtd,C} \times 10^{-3} + t_{i}
\end{equation*}
$$

$q_{rtd,C}$：定格冷房能力(W)    

In [ ]:
# 係数p_i (23)
def get_p_i_eq23(i, q_rtd_C):
    q_rtd_C = min(5600, q_rtd_C)
    s_i = get_s_i_eq23(i)
    t_i = get_t_i_eq23(i)
    return s_i * q_rtd_C * 10 ** (-3) + t_i


# 係数s_i (i=0,1,2,10..42)
def get_s_i_eq23(i):
    return table_5[4 - floor(i / 10)][(2 - (i % 10)) * 2]


# 係数t_i (i=0,1,2,10..42)
def get_t_i_eq23(i):
    return table_5[4 - floor(i / 10)][(2 - (i % 10)) * 2 + 1]

<div style="text-align: center;font-weight: bold;">表　係数$s_{i}$及び$t_{i}$</div>

|係数|係数|係数|係数|係数|係数|係数|係数|係数|係数|係数|係数|
|-|-|-|-|-|-|-|-|-|-|-|-|
|$s_{42}$|0.00000|$t_{42}$|0.00000|$s_{41}$|0.00000|$t_{41}$|0.00000|$s_{40}$|0.00000|$t_{40}$|0.00000|
|$s_{32}$|0.00000|$t_{32}$|0.00000|$s_{31}$|-0.00036|$t_{31}$|0.05080|$s_{30}$|-0.20346|$t_{30}$|0.47765|
|$s_{22}$|0.00000|$t_{22}$|0.00000|$s_{21}$|0.00227|$t_{21}$|-0.03952|$s_{20}$|0.04115|$t_{20}$|0.23099|
|$s_{12}$|0.00000|$t_{12}$|0.00000|$s_{11}$|-0.00911|$t_{11}$|0.07102|$s_{10}$|0.14950|$t_{10}$|-1.07335|
|$s_{02}$|0.00000|$t_{02}$|0.00000|$s_{01}$|0.00044|$t_{01}$|-0.00214|$s_{00}$|-0.06250|$t_{00}$|0.35150|


In [ ]:
# 表5 係数s_i及びt_i
table_5 = [
    (0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000),
    (0.00000, 0.00000, -0.00036, 0.05080, -0.20346, 0.47765),
    (0.00000, 0.00000, 0.00227, -0.03952, 0.04115, 0.23099),
    (0.00000, 0.00000, -0.00911, 0.07102, 0.14950, -1.07335),
    (0.00000, 0.00000, 0.00044, -0.00214, -0.06250, 0.35150)
]

容量可変型コンプレッサー搭載ルームエアコンディショナーについては下式により表される。

$q_{rtd,C} \leq 2200$ の場合

$$\large
p_{i} = p_{i,A}
$$

$2200 < q_{rtd,C} \leq 4000$ の場合

$$\large
p_{i} = p_{i,A} \times \frac{4000-q_{rtd,C}}{4000-2200} + p_{i,B} \times \frac{q_{rtd,C}-2200}{4000-2200}
$$

$4000 < q_{rtd,C} < 7100$ の場合

$$\large
p_{i} = p_{i,B} \times \frac{7100-q_{rtd,C}}{7100-4000} + p_{i,C} \times \frac{q_{rtd,C}-4000}{7100-4000}
$$

$7100 \leq q_{rtd,C}$ の場合

$$\large
p_{i} = p_{i,C}
$$

$p_{i,A}$：表により表される係数  
$p_{i,B}$：表により表される係数  
$p_{i,C}$：表により表される係数  

In [ ]:
# 係数p_i (21)
# (容量可変型コンプレッサー搭載ルームエアコンディショナー)
def get_p_i_eq24(i, q_rtd_C):
    if q_rtd_C <= 2200:
        # (9a)
        return get_p_i_A_eq24(i)
    elif 2200 < q_rtd_C and q_rtd_C <= 4000:
        # (9b)
        p_i_A = get_p_i_A_eq24(i)
        p_i_B = get_p_i_B_eq24(i)
        return p_i_A * ((4000 - q_rtd_C) / (4000 - 2200)) \
               + p_i_B * ((q_rtd_C - 2200) / (4000 - 2200))
    elif 4000 <= q_rtd_C and q_rtd_C < 7100:
        # (9c)
        p_i_B = get_p_i_B_eq24(i)
        p_i_C = get_p_i_C_eq24(i)
        return p_i_B * ((7100 - q_rtd_C) / (7100 - 4000)) \
               + p_i_C * ((q_rtd_C - 4000) / (7100 - 4000))
    elif 7100 <= q_rtd_C:
        return get_p_i_C_eq24(i)
    else:
        raise ValueError(q_rtd_C)


# 係数P_i_A (i=0,1,2,10..42)
def get_p_i_A_eq24(i):
    return table_6_A[4 - floor(i / 10)][(2 - (i % 10))]


def get_p_i_B_eq24(i):
    return table_6_B[4 - floor(i / 10)][(2 - (i % 10))]


def get_p_i_C_eq24(i):
    return table_6_C[4 - floor(i / 10)][(2 - (i % 10))]

<div style="text-align: center;font-weight: bold;">表　係数$p_{i,A}$</div>

|係数|値|係数|値|係数|値|
|-|-|-|-|-|-|
|$p_{42}$|-0.0004078|$p_{41}$|0.01035|$p_{40}$|-0.03248|
|$p_{32}$|0.0|$p_{31}$|0.04099|$p_{30}$|-0.81889|
|$p_{22}$|0.0|$p_{21}$|-0.04615|$p_{20}$|2.10666|
|$p_{12}$|0.0013382|$p_{11}$|-0.01179|$p_{10}$|-0.41778|
|$p_{02}$|0.0000000|$p_{01}$|-0.00102|$p_{00}$|0.09270|


In [ ]:
# 表6(A) 係数 p_i_A
table_6_A = [
    (-0.0004078, 0.01035, -0.03248),
    (0.0, 0.04099, -0.818809),
    (0.0, -0.04615, 2.10666),
    (0.0013382, -0.01179, -0.41778),
    (0.0000000, -0.00102, 0.09270)
]

<div style="text-align: center;font-weight: bold;">表　係数$p_{i,B}$</div>

|係数|値|係数|値|係数|値|
|-|-|-|-|-|-|
|$p_{42}$|-0.000056|$p_{41}$|-0.003539|$p_{40}$|-0.430566|
|$p_{32}$|0.0|$p_{31}$|0.015237|$p_{30}$|1.188850|
|$p_{22}$|0.0|$p_{21}$|0.000527|$p_{20}$|-0.304645|
|$p_{12}$|-0.000179|$p_{11}$|0.020543|$p_{10}$|0.130373|
|$p_{02}$|0.0|$p_{01}$|0.000240|$p_{00}$|0.013500|


In [ ]:
# 表6(B) 係数 p_i_B
table_6_B = [
    (-0.000056, -0.003539, -0.430566),
    (0.0, 0.015237, 1.188850),
    (0.0, 0.000527, -0.304645),
    (-0.000179, 0.020543, 0.130373),
    (0.0, 0.000240, 0.013500)
]

<div style="text-align: center;font-weight: bold;">表　係数$p_{i,C}$</div>

|係数|値|係数|値|係数|値|
|-|-|-|-|-|-|
|$p_{42}$|-0.0001598|$p_{41}$|0.004848|$p_{40}$|0.047097|
|$p_{32}$|0.0|$p_{31}$|0.016675|$p_{30}$|0.362141|
|$p_{22}$|0.0|$p_{21}$|-0.008134|$p_{20}$|-0.023535|
|$p_{12}$|-0.0000772|$p_{11}$|0.012558|$p_{10}$|0.056185|
|$p_{02}$|0.0|$p_{01}$|-0.000110|$p_{00}$|0.010300|


In [ ]:
# 表6(C) 係数 p_i_C
table_6_C = [
    (-0.0001598, 0.004848, 0.047097),
    (0.0, 0.016675, 0.362141),
    (0.0, -0.008134, -0.023535),
    (-0.0000772, 0.012558, 0.056185),
    (0.0, -0.000110, 0.010300)
]

$$\large
Q'_{T,C,d,t} = (Q_{T,CS,d,t} + Q_{T,CL,d,t}) \times \frac{1}{C_{hm,C} \times C_{af,C}}
$$

$Q_{T,CS,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器の処理冷房顕熱負荷(MJ/h)  
$Q_{T,CL,d,t}$：日付$d$の時刻$t$における1時間当たりの冷房設備機器の処理冷房潜熱負荷(MJ/h)  
$C_{hm,C}$：室内機吸い込み湿度に関する冷房出力補正係数  
$C_{af,C}$：室内機吹き出し風量に関する冷房出力補正係数  

In [ ]:
# 補正処理冷房負荷 (25)
def get_Q_dash_T_C_d_t(Q_T_CS_d_t, Q_T_CL_d_t):
    # 室内機吸い込み湿度に関する冷房出力補正係数
    C_hm_C = get_C_hm_C()

    # 室内機吹き出し風量に関する冷房出力係数
    C_af_C = get_C_af_C()

    return (Q_T_CS_d_t + Q_T_CL_d_t) * (1.0 / (C_hm_C * C_af_C))

#### 8.2 ガス消費量

日付$d$の時刻$t$における１時間当たりのガス消費量$E_{G,C,d,t}$は0とする。

In [ ]:
# ガス消費量
def get_E_G_C_d_t():
    return np.zeros(24 * 365)

#### 8.3 灯油消費量

日付$d$の時刻$t$における１時間当たりの灯油消費量$E_{K,C,d,t}$は0とする。

In [ ]:
# 灯油消費量
def get_E_K_C_d_t():
    return np.zeros(24 * 365)

#### 8.4 その他の燃料による一次エネルギー消費量

日付$d$の時刻$t$における１時間当たりのその他の燃料による一次エネルギー消費量$E_{M,C,d,t}$は0とする。

In [ ]:
# その他の燃料による一次エネルギー消費量
def get_E_M_C_d_t():
    return np.zeros(24 * 365)

### 付録 A 機器の性能を表す仕様の決定方法

#### A.1 記号と単位

省略

#### A.2 定格能力

$$
\begin{equation*}
\large
q_{rtd,H} = 1.2090 \times q_{rtd,C} - 85.1
\end{equation*}
$$

$q_{rtd,H}$：定格暖房能力(W)    
$q_{rtd,C}$：定格冷房能力(W)    

In [ ]:
# 定格暖房能力 (1)
def get_q_rtd_H(q_rtd_C):
    return 1.2090 * q_rtd_C - 85.1

定格冷房能力$q_{rtd,C}$が5600(W)を超える場合については、定格冷房能力$q_{rtd,C}$は5600(W)に等しいとして計算するものとする。

$$
\begin{equation*}
\large
q_{rtd,C} = 190.5 \times A_{HCZ} + 45.6
\end{equation*}
$$

$A_{HCZ}$：ルームエアコンディショナーにより暖冷房される暖冷房区画の床面積(m<sup>2</sup>)

In [ ]:
# 定格冷房能力 (2)
def get_q_rtd_C(A_HCZ):
    q_rtd_C = 190.5 * A_HCZ + 45.6
    return min(5600, q_rtd_C)

#### A.3 最大能力

$$
\begin{equation*}
\large
q_{max,H} = 1.7597 \times q_{max,C} - 413.7
\end{equation*}
$$

$q_{max,H}$：最大暖房能力(W)    
$q_{max,C}$：最大冷房能力(W)    

In [ ]:
# 最大暖房能力 (3)
def get_q_max_H(q_rtd_H, q_max_C):
    q_max_H = 1.7597 * q_max_C - 413.7
    return max(q_rtd_H, q_max_H)

定格暖房能力$q_{rtd,H}$が最大暖房能力$q_{max,H}$を超える場合については、最大暖房能力$q_{max,H}$は定格暖房能力$q_{rtd,H}$に等しいとする。

$$
\begin{equation*}
\large
q_{max,C} = 0.8462 \times q_{rtd,C} + 1205.9
\end{equation*}
$$

$q_{rtd,C}$：定格冷房能力(W)    

定格冷房能力$q_{rtd,C}$が最大冷房能力$q_{max,C}$を超える場合については、最大冷房能力$q_{max,C}$は定格冷房能力$q_{rtd,C}$に等しいとする。

In [ ]:
# 最大冷房能力 (4)
def get_q_max_C(q_rtd_C):
    """
    :param q_rtd_C:  定格冷房能力
    :return:
    """
    q_max_C = 0.8462 * q_rtd_C + 1205.9
    return max(q_rtd_C, q_max_C)

#### A.4 定格エネルギー消費効率

$$
\begin{equation*}
\large
e_{rtd,H} = 0.77 \times e_{rtd,C} + 1.66
\end{equation*}
$$

$e_{rtd,H}$：定格暖房エネルギー消費効率    
$e_{rtd,C}$：定格冷房エネルギー消費効率    

In [ ]:
# 定格暖房エネルギー効率 (5)
def get_e_rtd_H(e_rtd_C):
    return 0.77 * e_rtd_C + 1.66

区分（い）

$$
\begin{equation*}
\large
e_{rtd,C} = −0.553 \times 10^{−3} \times q_{rtd,C} + 6.34
\end{equation*}
$$

区分（ろ）

$$
\begin{equation*}
\large
e_{rtd,C} = −0.504 \times 10^{−3} \times q_{rtd,C} + 5.88
\end{equation*}
$$

区分（は）

$$
\begin{equation*}
\large
e_{rtd,C} = −0.473 \times 10^{−3} \times q_{rtd,C} + 5.50
\end{equation*}
$$

$q_{rtd,C}$：定格冷房能力(W)    

In [ ]:
# 定格冷房エネルギー効率 (6)
def get_e_rtd_C(e_class, q_rtd_C):
    if e_class == 'い':
        # (6a)
        return -0.553 * 10 ** (-3) * q_rtd_C + 6.34
    elif e_class == 'ろ':
        # (6b)
        return -0.504 * 10 ** (-3) * q_rtd_C + 5.88
    elif e_class == 'は' or e_class == '不明' or e_class == None:
        # (6c)
        return -0.473 * 10 ** (-3) * q_rtd_C + 5.50
    else:
        raise ValueError(e_class)

#### A.5 容量可変型コンプレッサー搭載の有無

省略

#### A.6 複数のルームエアコンディショナーが設置される場合の仕様の決定方法

省略

### 付録 B 容量可変型コンプレッサー搭載ルームエアコンディショナーの定義と判別方法

省略

### 付録 C （参考）可変気筒方式コンプレッサー搭載ルームエアコンディショナーの判別方法

省略